# **프로젝트 #2**


---



### **팀소개**

*   팀 번호 : 팀 19
*   팀원 : 김용호


### **프로젝트 목표**

*   네트워크 악성 플로우를 통해 악성 외부 IP 탐지



### **프로젝트 내용**

*   악성 플로우를 탐지 할 수 있는 새로운 feature를 생성하여 악성 탐지 모델을 생성 후 악성IP 탐지


### **프로젝트 데이터 셋**

*   국민대학교 네트워크 플로우 데이터셋 (5시간)   
*   네트워크 플로우 : 한번의 세션에서 출발지(Source IP)와 도착지(Destination IP) 사이에 오가는 패킷에 대한 통계적 데이터


### **제출 형식**

*   Ipynb : run all 하였을 경우 동일한 csv가 생성되는 전체코드 제출
*   csv 파일 :파일 이름 : <팀 번호>_Project2.csv











# 프로젝트 설정

---



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/4학년/2023-2정보보호와시스템보안_최종데이터"

/content/drive/MyDrive/4학년/2023-2정보보호와시스템보안_최종데이터


In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
def open_pickle(path: str):
    with open(path, 'rb') as f:
        target = pickle.load(f)
    return target

# 1. train, valid 데이터 셋 불러오기

In [5]:
train_path = r"./train.csv"
valid_path = r"./valid.csv"

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

In [6]:
train_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,03:52:00,3119967,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,03:37:20,2991830,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
2,21.122.91.165,25918,30.239.23.122,53,TCP,......S.,03:10:28,3004658,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
3,173.123.45.117,4445,30.239.23.122,53,TCP,...A...F,00:42:24,4402645,2,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
4,21.122.91.165,28467,30.239.23.122,53,TCP,......S.,02:45:28,3001784,3,0,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977265,161.208.37.158,5353,100.144.85.150,5353,UDP,........,02:27:22,96782806,7512,0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
977266,184.168.154.121,0,100.144.85.150,0,0,........,02:27:22,98654357,1677,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
977267,100.144.85.150,6672,173.123.38.32,5353,UDP,........,02:27:21,97236104,7550,7545,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP
977268,21.122.97.241,5353,100.144.85.150,6672,UDP,........,02:27:19,99573401,11707,0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP


# 2. feature 추가하기

In [7]:
# 시간별로 분리
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'])
valid_df['Timestamp'] = pd.to_datetime(valid_df['Timestamp'])
train_df.dtypes

Src IP                                object
Src Port                               int64
Dst IP                                object
Dst Port                               int64
Protocol                              object
Flags                                 object
Timestamp                     datetime64[ns]
Flow Duration                          int64
Total Fwd Packet                       int64
Total Bwd packets                      int64
Total Length of Fwd Packet           float64
Total Length of Bwd Packet           float64
Fwd Packet Length Max                float64
Fwd Packet Length Min                float64
Fwd Packet Length Std                float64
Bwd Packet Length Max                float64
Bwd Packet Length Min                float64
Bwd Packet Length Std                float64
Flow IAT Mean                        float64
Flow IAT Std                         float64
Flow IAT Max                         float64
Flow IAT Min                         float64
Fwd IAT To

In [8]:
train_df['hour'] = train_df['Timestamp'].dt.hour
valid_df['hour'] = valid_df['Timestamp'].dt.hour
train_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,hour
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,2023-12-01 03:52:00,3119967,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,2023-12-01 03:37:20,2991830,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3
2,21.122.91.165,25918,30.239.23.122,53,TCP,......S.,2023-12-01 03:10:28,3004658,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3
3,173.123.45.117,4445,30.239.23.122,53,TCP,...A...F,2023-12-01 00:42:24,4402645,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,0
4,21.122.91.165,28467,30.239.23.122,53,TCP,......S.,2023-12-01 02:45:28,3001784,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977265,161.208.37.158,5353,100.144.85.150,5353,UDP,........,2023-12-01 02:27:22,96782806,7512,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2
977266,184.168.154.121,0,100.144.85.150,0,0,........,2023-12-01 02:27:22,98654357,1677,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2
977267,100.144.85.150,6672,173.123.38.32,5353,UDP,........,2023-12-01 02:27:21,97236104,7550,7545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2
977268,21.122.97.241,5353,100.144.85.150,6672,UDP,........,2023-12-01 02:27:19,99573401,11707,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2


In [9]:
# Src 시간별 entity 분리
entity_list = []
for idx, row in train_df.groupby(['Src IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
train_df['s_entity'] = ['_'.join([str(x) for x in idx]) for idx in train_df[['Src IP', 'hour']].itertuples(index=False)]


In [10]:
entity_list = []
for idx, row in valid_df.groupby(['Src IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
valid_df['s_entity'] = ['_'.join([str(x) for x in idx]) for idx in valid_df[['Src IP', 'hour']].itertuples(index=False)]

In [11]:
# Dst 시간별 entity 분리
entity_list = []
for idx, row in train_df.groupby(['Dst IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
train_df['d_entity'] = ['_'.join([str(x) for x in idx]) for idx in train_df[['Dst IP', 'hour']].itertuples(index=False)]


In [12]:
entity_list = []
for idx, row in valid_df.groupby(['Dst IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
valid_df['d_entity'] = ['_'.join([str(x) for x in idx]) for idx in valid_df[['Dst IP', 'hour']].itertuples(index=False)]


In [13]:
train_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,hour,s_entity,d_entity
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,2023-12-01 03:52:00,3119967,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3,21.122.91.166_3,30.239.23.122_3
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,2023-12-01 03:37:20,2991830,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3,21.122.91.166_3,30.239.23.122_3
2,21.122.91.165,25918,30.239.23.122,53,TCP,......S.,2023-12-01 03:10:28,3004658,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,3,21.122.91.165_3,30.239.23.122_3
3,173.123.45.117,4445,30.239.23.122,53,TCP,...A...F,2023-12-01 00:42:24,4402645,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,0,173.123.45.117_0,30.239.23.122_0
4,21.122.91.165,28467,30.239.23.122,53,TCP,......S.,2023-12-01 02:45:28,3001784,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2,21.122.91.165_2,30.239.23.122_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977265,161.208.37.158,5353,100.144.85.150,5353,UDP,........,2023-12-01 02:27:22,96782806,7512,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2,161.208.37.158_2,100.144.85.150_2
977266,184.168.154.121,0,100.144.85.150,0,0,........,2023-12-01 02:27:22,98654357,1677,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2,184.168.154.121_2,100.144.85.150_2
977267,100.144.85.150,6672,173.123.38.32,5353,UDP,........,2023-12-01 02:27:21,97236104,7550,7545,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2,100.144.85.150_2,173.123.38.32_2
977268,21.122.97.241,5353,100.144.85.150,6672,UDP,........,2023-12-01 02:27:19,99573401,11707,0,...,0.0,0.0,0.0,0.0,0.0,0.0,Benign_IP,2,21.122.97.241_2,100.144.85.150_2


In [14]:
# 시간당 요청수 계산
req_cnt_per_hour = train_df.groupby('s_entity').size()
train_df = pd.merge(train_df, req_cnt_per_hour.rename('req_cnt_per_hour'), left_on='s_entity', right_index=True)

In [15]:
req_cnt_per_hour = valid_df.groupby('s_entity').size()
valid_df = pd.merge(valid_df, req_cnt_per_hour.rename('req_cnt_per_hour'), left_on='s_entity', right_index=True)

In [16]:
# 시간당 응답수 계산
res_cnt_per_hour = train_df.groupby('d_entity').size()
train_df = pd.merge(train_df, res_cnt_per_hour.rename('res_cnt_per_hour'), left_on='d_entity', right_index=True)

In [17]:
res_cnt_per_hour = valid_df.groupby('d_entity').size()
valid_df = pd.merge(valid_df, res_cnt_per_hour.rename('res_cnt_per_hour'), left_on='d_entity', right_index=True)

In [19]:
# 시간당 요청속도 계산
train_df['req_freq_per_hour'] = 0

for entity in train_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = train_df[(train_df['Src IP'] == ip) & (train_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        train_df.loc[group.index, 'req_freq_per_hour'] = np.mean(delta)
    else:
        train_df.loc[group.index, 'req_freq_per_hour'] = np.nan

In [20]:
valid_df['req_freq_per_hour'] = 0

for entity in valid_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = valid_df[(valid_df['Src IP'] == ip) & (valid_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        valid_df.loc[group.index, 'req_freq_per_hour'] = np.mean(delta)
    else:
        valid_df.loc[group.index, 'req_freq_per_hour'] = np.nan

In [21]:
# 시간당 응답속도 계산
train_df['res_freq_per_hour'] = 0

for entity in train_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = train_df[(train_df['Dst IP'] == ip) & (train_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        train_df.loc[group.index, 'res_freq_per_hour'] = np.mean(delta)
    else:
        train_df.loc[group.index, 'res_freq_per_hour'] = np.nan

In [22]:
valid_df['res_freq_per_hour'] = 0

for entity in valid_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = valid_df[(valid_df['Dst IP'] == ip) & (valid_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        valid_df.loc[group.index, 'res_freq_per_hour'] = np.mean(delta)
    else:
        valid_df.loc[group.index, 'res_freq_per_hour'] = np.nan

In [24]:
# 시간당 요청 균일성 계산
train_df['req_even_per_hour'] = 0

for entity in train_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = train_df[(train_df['Src IP'] == ip) & (train_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        train_df.loc[group.index, 'req_even_per_hour'] = np.std(delta)
    else:
        train_df.loc[group.index, 'req_even_per_hour'] = np.nan

In [25]:
valid_df['req_even_per_hour'] = 0

for entity in valid_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = valid_df[(valid_df['Src IP'] == ip) & (valid_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        valid_df.loc[group.index, 'req_even_per_hour'] = np.std(delta)
    else:
        valid_df.loc[group.index, 'req_even_per_hour'] = np.nan

In [26]:
# 시간당 응답 균일성 계산
train_df['res_even_per_hour'] = 0

for entity in train_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = train_df[(train_df['Dst IP'] == ip) & (train_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        train_df.loc[group.index, 'res_even_per_hour'] = np.std(delta)
    else:
        train_df.loc[group.index, 'res_even_per_hour'] = np.nan

In [27]:
valid_df['res_even_per_hour'] = 0

for entity in valid_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = valid_df[(valid_df['Dst IP'] == ip) & (valid_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        valid_df.loc[group.index, 'res_even_per_hour'] = np.std(delta)
    else:
        valid_df.loc[group.index, 'res_even_per_hour'] = np.nan

In [28]:
# 결측치 처리
train_df.fillna(0, inplace=True)
valid_df.fillna(0, inplace=True)

In [29]:
train_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Label,hour,s_entity,d_entity,req_cnt_per_hour,res_cnt_per_hour,req_freq_per_hour,res_freq_per_hour,req_even_per_hour,res_even_per_hour
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,2023-12-01 03:52:00,3119967,3,0,...,Benign_IP,3,21.122.91.166_3,30.239.23.122_3,800,54,-0.996245,-45.509434,1463.305187,1582.216831
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,2023-12-01 03:37:20,2991830,3,0,...,Benign_IP,3,21.122.91.166_3,30.239.23.122_3,800,54,-0.996245,-45.509434,1463.305187,1582.216831
12,21.122.91.166,53971,30.239.23.122,53,TCP,......S.,2023-12-01 03:32:34,2998037,3,0,...,Benign_IP,3,21.122.91.166_3,30.239.23.122_3,800,54,-0.996245,-45.509434,1463.305187,1582.216831
13,21.122.91.166,31334,30.239.23.122,53,TCP,......S.,2023-12-01 03:46:36,2993751,3,0,...,Benign_IP,3,21.122.91.166_3,30.239.23.122_3,800,54,-0.996245,-45.509434,1463.305187,1582.216831
30,21.122.91.166,42301,30.239.23.122,53,TCP,......S.,2023-12-01 03:55:51,3002045,3,0,...,Benign_IP,3,21.122.91.166_3,30.239.23.122_3,800,54,-0.996245,-45.509434,1463.305187,1582.216831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974347,82.144.123.25,8082,30.17.111.51,1900,UDP,........,2023-12-01 05:00:55,1622,44,0,...,Attack_IP,5,82.144.123.25_5,30.17.111.51_5,3,1,2.000000,0.000000,24.000000,0.000000
974352,82.144.123.25,8082,30.17.61.224,30301,UDP,........,2023-12-01 05:00:33,0,2,0,...,Attack_IP,5,82.144.123.25_5,30.17.61.224_5,3,1,2.000000,0.000000,24.000000,0.000000
974711,82.38.47.202,54578,112.21.100.186,27555,TCP,......S.,2023-12-01 05:00:41,0,2,0,...,Attack_IP,5,82.38.47.202_5,112.21.100.186_5,11,1,2.300000,0.000000,26.705992,0.000000
974846,82.38.47.202,54578,30.17.111.255,29730,TCP,......S.,2023-12-01 05:01:06,0,2,0,...,Attack_IP,5,82.38.47.202_5,30.17.111.255_5,11,1,2.300000,0.000000,26.705992,0.000000


In [30]:
# ip 비율
train_sp_count = train_df['Src IP'].value_counts(normalize=True).to_dict()
train_dp_count = train_df['Dst IP'].value_counts(normalize=True).to_dict()
valid_sp_count = valid_df['Src IP'].value_counts(normalize=True).to_dict()
valid_dp_count = valid_df['Dst IP'].value_counts(normalize=True).to_dict()

train_df['src_ip_count'] = train_df['Src IP'].map(train_sp_count)
train_df['dst_ip_count'] = train_df['Dst IP'].map(train_dp_count)
valid_df['src_ip_count'] = valid_df['Src IP'].map(valid_sp_count)
valid_df['dst_ip_count'] = valid_df['Dst IP'].map(valid_dp_count)

In [31]:
# port 비율
train_sp_count = train_df['Src Port'].value_counts(normalize=True).to_dict()
train_dp_count = train_df['Dst Port'].value_counts(normalize=True).to_dict()
valid_sp_count = valid_df['Src Port'].value_counts(normalize=True).to_dict()
valid_dp_count = valid_df['Dst Port'].value_counts(normalize=True).to_dict()

train_df['src_port_count'] = train_df['Src Port'].map(train_sp_count)
train_df['dst_port_count'] = train_df['Dst Port'].map(train_dp_count)
valid_df['src_port_count'] = valid_df['Src Port'].map(valid_sp_count)
valid_df['dst_port_count'] = valid_df['Dst Port'].map(valid_dp_count)

In [32]:
# standardScaler
from sklearn.preprocessing import StandardScaler

columns_to_scale = train_df.select_dtypes(include=['int64','float64']).columns.drop(['Src Port', 'Dst Port',
        'hour'])
normalizer = StandardScaler()
normalizer = normalizer.fit(train_df[columns_to_scale])
train_df[columns_to_scale] = normalizer.transform(train_df[columns_to_scale])
train_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,req_cnt_per_hour,res_cnt_per_hour,req_freq_per_hour,res_freq_per_hour,req_even_per_hour,res_even_per_hour,src_ip_count,dst_ip_count,src_port_count,dst_port_count
0,21.122.91.166,13850,30.239.23.122,53,TCP,......S.,2023-12-01 03:52:00,-0.313958,-0.127507,-0.028561,...,-0.712923,-0.579435,-0.035906,-0.725442,0.772587,1.042989,-0.762894,-0.597181,-0.707584,0.184102
1,21.122.91.166,43842,30.239.23.122,53,TCP,......S.,2023-12-01 03:37:20,-0.319044,-0.127507,-0.028561,...,-0.712923,-0.579435,-0.035906,-0.725442,0.772587,1.042989,-0.762894,-0.597181,-0.707288,0.184102
12,21.122.91.166,53971,30.239.23.122,53,TCP,......S.,2023-12-01 03:32:34,-0.318797,-0.127507,-0.028561,...,-0.712923,-0.579435,-0.035906,-0.725442,0.772587,1.042989,-0.762894,-0.597181,-0.707354,0.184102
13,21.122.91.166,31334,30.239.23.122,53,TCP,......S.,2023-12-01 03:46:36,-0.318968,-0.127507,-0.028561,...,-0.712923,-0.579435,-0.035906,-0.725442,0.772587,1.042989,-0.762894,-0.597181,-0.707617,0.184102
30,21.122.91.166,42301,30.239.23.122,53,TCP,......S.,2023-12-01 03:55:51,-0.318638,-0.127507,-0.028561,...,-0.712923,-0.579435,-0.035906,-0.725442,0.772587,1.042989,-0.762894,-0.597181,-0.707552,0.184102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974347,82.144.123.25,8082,30.17.111.51,1900,UDP,........,2023-12-01 05:00:55,-0.437723,-0.007056,-0.028561,...,-0.840349,-0.582724,0.023730,-0.074389,-2.522538,-2.949044,-0.818781,-0.596840,-0.702059,-0.709842
974352,82.144.123.25,8082,30.17.61.224,30301,UDP,........,2023-12-01 05:00:33,-0.437787,-0.130445,-0.028561,...,-0.840349,-0.582724,0.023730,-0.074389,-2.522538,-2.949044,-0.818781,-0.596186,-0.702059,-0.721580
974711,82.38.47.202,54578,112.21.100.186,27555,TCP,......S.,2023-12-01 05:00:41,-0.437787,-0.130445,-0.028561,...,-0.839070,-0.582724,0.029701,-0.074389,-2.516343,-2.949044,-0.743381,-0.596696,-0.627428,-0.730011
974846,82.38.47.202,54578,30.17.111.255,29730,TCP,......S.,2023-12-01 05:01:06,-0.437787,-0.130445,-0.028561,...,-0.839070,-0.582724,0.029701,-0.074389,-2.516343,-2.949044,-0.743381,-0.597207,-0.627428,-0.729458


In [33]:
columns_to_scale = valid_df.select_dtypes(include=['int64','float64']).columns.drop(['Src Port', 'Dst Port',
        'hour'])
normalizer = StandardScaler()
normalizer = normalizer.fit(valid_df[columns_to_scale])
valid_df[columns_to_scale] = normalizer.transform(valid_df[columns_to_scale])
valid_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,req_cnt_per_hour,res_cnt_per_hour,req_freq_per_hour,res_freq_per_hour,req_even_per_hour,res_even_per_hour,src_ip_count,dst_ip_count,src_port_count,dst_port_count
0,173.123.45.54,61650,30.239.23.122,53,TCP,...A...F,2023-12-01 00:47:49,-0.261560,-0.127211,-0.023046,...,-0.677374,-0.581396,-0.021032,0.283253,0.132413,1.541031,-0.784406,-0.597472,-0.708138,0.184025
4,173.123.45.54,62593,30.239.23.122,53,TCP,...A...F,2023-12-01 00:53:13,-0.258374,-0.127211,-0.023046,...,-0.677374,-0.581396,-0.021032,0.283253,0.132413,1.541031,-0.784406,-0.597472,-0.707742,0.184025
9,173.123.45.54,52414,30.239.23.122,53,TCP,...A...F,2023-12-01 00:03:25,-0.263517,-0.127211,-0.023046,...,-0.677374,-0.581396,-0.021032,0.283253,0.132413,1.541031,-0.784406,-0.597472,-0.707248,0.184025
20,173.123.45.54,60930,30.239.23.122,53,TCP,...A...F,2023-12-01 00:42:55,-0.258703,-0.127211,-0.023046,...,-0.677374,-0.581396,-0.021032,0.283253,0.132413,1.541031,-0.784406,-0.597472,-0.706753,0.184025
39,173.123.45.54,52694,30.239.23.122,53,TCP,...A...F,2023-12-01 00:04:23,-0.259895,-0.127211,-0.023046,...,-0.677374,-0.581396,-0.021032,0.283253,0.132413,1.541031,-0.784406,-0.597472,-0.707742,0.184025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324552,82.101.43.184,54945,173.123.32.128,2323,TCP,......S.,2023-12-01 00:57:59,0.410539,-0.127211,-0.029937,...,-0.840506,-0.582700,9.113799,-0.115841,2.537712,-2.872233,-0.820796,-0.599394,-0.706654,-0.726963
324567,82.147.101.52,0,30.17.110.253,0,0,........,2023-12-01 05:00:44,-0.359007,0.565430,-0.029937,...,-0.841465,-0.582700,-0.019674,-0.115841,-2.802035,-2.872233,-0.820893,-0.596412,2.179067,0.423646
325424,161.208.37.158,5353,102.215.171.4,5353,UDP,........,2023-12-01 02:25:43,3.466224,20.600294,-0.029937,...,-0.841465,-0.582700,-0.019674,-0.115841,-2.802035,-2.872233,-0.821864,-0.599394,-0.700028,-0.715148
325518,100.131.35.240,0,173.123.43.244,0,0,........,2023-12-01 01:31:39,-0.438008,-0.130085,-0.023046,...,-0.831869,-0.582700,-0.407216,-0.115841,0.752494,-2.872233,-0.810413,-0.599002,2.179067,0.423646


In [34]:
train_df = pd.get_dummies(train_df, columns=['Protocol'])
valid_df = pd.get_dummies(valid_df, columns=['Protocol'])

In [35]:
train_df['Has_SYN'] = train_df['Flags'].apply(lambda x: 1 if 'S' in x else 0)
train_df['Has_FIN'] = train_df['Flags'].apply(lambda x: 1 if 'F' in x else 0)
train_df['Has_ACK'] = train_df['Flags'].apply(lambda x: 1 if 'A' in x else 0)
train_df['Has_RST'] = train_df['Flags'].apply(lambda x: 1 if 'R' in x else 0)
train_df['Has_PSH'] = train_df['Flags'].apply(lambda x: 1 if 'P' in x else 0)
train_df['Has_URG'] = train_df['Flags'].apply(lambda x: 1 if 'U' in x else 0)
train_df['Has_ECE'] = train_df['Flags'].apply(lambda x: 1 if 'E' in x else 0)
train_df['Has_CWR'] = train_df['Flags'].apply(lambda x: 1 if 'C' in x else 0)
train_df['Has_Not_Flags'] = train_df['Flags'].apply(lambda x: 1 if 'S' not in x and 'F' not in x and 'A' not in x and 'R' not in x and 'P' not in x and 'U' not in x and 'E' not in x and 'C' not in x else 0)

In [36]:
valid_df['Has_SYN'] = valid_df['Flags'].apply(lambda x: 1 if 'S' in x else 0)
valid_df['Has_FIN'] = valid_df['Flags'].apply(lambda x: 1 if 'F' in x else 0)
valid_df['Has_ACK'] = valid_df['Flags'].apply(lambda x: 1 if 'A' in x else 0)
valid_df['Has_RST'] = valid_df['Flags'].apply(lambda x: 1 if 'R' in x else 0)
valid_df['Has_PSH'] = valid_df['Flags'].apply(lambda x: 1 if 'P' in x else 0)
valid_df['Has_URG'] = valid_df['Flags'].apply(lambda x: 1 if 'U' in x else 0)
valid_df['Has_ECE'] = valid_df['Flags'].apply(lambda x: 1 if 'E' in x else 0)
valid_df['Has_CWR'] = valid_df['Flags'].apply(lambda x: 1 if 'C' in x else 0)
valid_df['Has_Not_Flags'] = valid_df['Flags'].apply(lambda x: 1 if 'S' not in x and 'F' not in x and 'A' not in x and 'R' not in x and 'P' not in x and 'U' not in x and 'E' not in x and 'C' not in x else 0)

In [37]:
train_df['dst_port_same_src_port'] = (train_df['Dst Port'] == train_df['Src Port']).astype(int)
valid_df['dst_port_same_src_port'] = (valid_df['Dst Port'] == valid_df['Src Port']).astype(int)

In [38]:
train_y = [0 if i == "Benign_IP" else 1 for i in train_df['Label']]
valid_y = [0 if i == "Benign_IP" else 1 for i in valid_df['Label']]

In [39]:
train_expt = train_df[[
       'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'req_cnt_per_hour',
       'res_cnt_per_hour', 'req_freq_per_hour', 'res_freq_per_hour',
       'req_even_per_hour', 'res_even_per_hour', 'src_ip_count',
       'dst_ip_count','src_port_count',
       'dst_port_count', 'Protocol_TCP', 'Protocol_UDP',
       'Has_SYN', 'Has_FIN', 'Has_ACK', 'Has_RST', 'Has_PSH', 'Has_URG',
       'Has_ECE', 'Has_CWR', 'Has_Not_Flags', 'dst_port_same_src_port']]
train_list = train_expt.values.tolist()

valid_expt = valid_df[[
       'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'req_cnt_per_hour',
       'res_cnt_per_hour', 'req_freq_per_hour', 'res_freq_per_hour',
       'req_even_per_hour', 'res_even_per_hour', 'src_ip_count',
       'dst_ip_count','src_port_count',
       'dst_port_count', 'Protocol_TCP', 'Protocol_UDP',
       'Has_SYN', 'Has_FIN', 'Has_ACK', 'Has_RST', 'Has_PSH', 'Has_URG',
       'Has_ECE', 'Has_CWR', 'Has_Not_Flags', 'dst_port_same_src_port']]
valid_list = valid_expt.values.tolist()

In [40]:
#학습을 위한 데이터 타입 변환

train_X = np.array(train_list, dtype=np.float32)
valid_X = np.array(valid_list, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)
valid_y = np.array(valid_y, dtype=np.float32)

# 3. 모델 학습

In [41]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 108) # randomstate는 108로 고정
rfc.fit(train_X, train_y)
y_pred = rfc.predict(valid_X)

플로우 기반 이상탐지 성능평가

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 성능 평가를 위해 예측 값을 반올림
round_valid_y = valid_y.astype("int")
round_y_pred = np.round(y_pred)
print("confusion_matrix\n", confusion_matrix(round_valid_y, round_y_pred))
print(f"accuracy\t: {accuracy_score(round_valid_y, round_y_pred): .5f}")
print(f"precision\t: {precision_score(round_valid_y, round_y_pred): .5f}")
print(f"recall\t\t: {recall_score(round_valid_y, round_y_pred): .5f}")
print(f"F-1\t\t: {f1_score(round_valid_y, round_y_pred): .5f}")

confusion_matrix
 [[186663   3610]
 [   901 134386]]
accuracy	:  0.98614
precision	:  0.97384
recall		:  0.99334
F-1		:  0.98349


# 4. IP voting

실제 성능평가는 IP기준으로 해야하기 때문에 flow 기반 예측결과를 이용하여 IP votiong을 진행합니다.

In [43]:
# 정상 외부 IP set (정답)
outer_benignIP_set = open_pickle("./outer_benign_IP_train.pkl")
# 악성 외부 IP set (정답)
outer_malIP_set = open_pickle("./outer_mal_IP_train.pkl")

In [44]:
#외부 IP의 악성, 정상 개수 확인
len(outer_benignIP_set), len(outer_malIP_set)

(590, 457)

In [45]:
# IP 탐지를 위해 dictionary형태로 변경
labelIP_dict = dict()

for ip in outer_benignIP_set:
    labelIP_dict[ip] = 0
for ip in outer_malIP_set:
    labelIP_dict[ip] = 1

In [46]:
predictIP_dict = {}

for idx, data in enumerate(valid_df.values):
    if data[0] in labelIP_dict:
        if data[0] not in predictIP_dict:
            predictIP_dict[data[0]] = 0

        if y_pred[idx] == 1:
            predictIP_dict[data[0]] = 1

    else:
        if data[2] not in predictIP_dict:
            predictIP_dict[data[2]] = 0

        if y_pred[idx] == 1:
            predictIP_dict[data[2]] = 1

In [47]:
pred_IP = []
label_IP = []

for k in predictIP_dict.keys():
    pred_IP.append(1 if predictIP_dict[k] == 1 else 0)
    label_IP.append(labelIP_dict[k])

print("confusion_matrix\n", confusion_matrix(label_IP, pred_IP))
print(f"accuracy\t: {accuracy_score(label_IP, pred_IP): .5f}")
print(f"precision\t: {precision_score(label_IP, pred_IP): .5f}")
print(f"recall\t\t: {recall_score(label_IP, pred_IP): .5f}")
print(f"F-1\t\t: {f1_score(label_IP, pred_IP): .5f}")

confusion_matrix
 [[538  52]
 [  4 453]]
accuracy	:  0.94651
precision	:  0.89703
recall		:  0.99125
F-1		:  0.94179


# 5. 제출 csv 생성

In [48]:
# test.csv가 제공되면 실행하면 됩니다!
test_df = pd.read_csv("./2023-2informationSecurity_testdataset/test.csv")

In [50]:
# 시간별로 분리
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'])
test_df.dtypes

Src IP                                object
Src Port                               int64
Dst IP                                object
Dst Port                               int64
Protocol                              object
Flags                                 object
Timestamp                     datetime64[ns]
Flow Duration                          int64
Total Fwd Packet                       int64
Total Bwd packets                      int64
Total Length of Fwd Packet           float64
Total Length of Bwd Packet           float64
Fwd Packet Length Max                float64
Fwd Packet Length Min                float64
Fwd Packet Length Std                float64
Bwd Packet Length Max                float64
Bwd Packet Length Min                float64
Bwd Packet Length Std                float64
Flow IAT Mean                        float64
Flow IAT Std                         float64
Flow IAT Max                         float64
Flow IAT Min                         float64
Fwd IAT To

In [51]:
test_df['hour'] = test_df['Timestamp'].dt.hour

In [52]:
# Src 시간별 entity 분리
entity_list = []
for idx, row in test_df.groupby(['Src IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
test_df['s_entity'] = ['_'.join([str(x) for x in idx]) for idx in test_df[['Src IP', 'hour']].itertuples(index=False)]


In [53]:
# Dst 시간별 entity 분리
entity_list = []
for idx, row in test_df.groupby(['Dst IP', 'hour']).first().iterrows():
    entity_list.append('_'.join([str(x) for x in idx]))
test_df['d_entity'] = ['_'.join([str(x) for x in idx]) for idx in test_df[['Dst IP', 'hour']].itertuples(index=False)]


In [54]:
# 시간당 요청수 계산
req_cnt_per_hour = test_df.groupby('s_entity').size()
test_df = pd.merge(test_df, req_cnt_per_hour.rename('req_cnt_per_hour'), left_on='s_entity', right_index=True)

In [55]:
# 시간당 응답수 계산
res_cnt_per_hour = test_df.groupby('d_entity').size()
test_df = pd.merge(test_df, res_cnt_per_hour.rename('res_cnt_per_hour'), left_on='d_entity', right_index=True)

In [56]:
# 시간당 요청속도 계산
test_df['req_freq_per_hour'] = 0

for entity in test_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = test_df[(test_df['Src IP'] == ip) & (test_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        test_df.loc[group.index, 'req_freq_per_hour'] = np.mean(delta)
    else:
        test_df.loc[group.index, 'req_freq_per_hour'] = np.nan

In [57]:
# 시간당 응답속도 계산
test_df['res_freq_per_hour'] = 0

for entity in test_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = test_df[(test_df['Dst IP'] == ip) & (test_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        test_df.loc[group.index, 'res_freq_per_hour'] = np.mean(delta)
    else:
        test_df.loc[group.index, 'res_freq_per_hour'] = np.nan

In [58]:
# 시간당 요청 균일성 계산
test_df['req_even_per_hour'] = 0

for entity in test_df['s_entity'].unique():
    ip, hour = entity.split('_')
    group = test_df[(test_df['Src IP'] == ip) & (test_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        test_df.loc[group.index, 'req_even_per_hour'] = np.std(delta)
    else:
        test_df.loc[group.index, 'req_even_per_hour'] = np.nan

In [59]:
# 시간당 응답 균일성 계산
test_df['res_even_per_hour'] = 0

for entity in test_df['d_entity'].unique():
    ip, hour = entity.split('_')
    group = test_df[(test_df['Dst IP'] == ip) & (test_df['hour'] == int(hour))]

    delta = []
    prev_row = group.iloc[0, :]

    for idx, row in group[1:].iterrows():
        tdelta = row['Timestamp'] - prev_row['Timestamp']
        delta.append(tdelta.total_seconds())
        prev_row = row

    if len(delta) > 0:
        test_df.loc[group.index, 'res_even_per_hour'] = np.std(delta)
    else:
        test_df.loc[group.index, 'res_even_per_hour'] = np.nan

In [60]:
# 결측치 처리
test_df.fillna(0, inplace=True)

In [61]:
# ip 비율
test_sp_count = test_df['Src IP'].value_counts(normalize=True).to_dict()
test_dp_count = test_df['Dst IP'].value_counts(normalize=True).to_dict()

test_df['src_ip_count'] = test_df['Src IP'].map(test_sp_count)
test_df['dst_ip_count'] = test_df['Dst IP'].map(test_dp_count)

In [62]:
# port 비율
test_sp_count = test_df['Src Port'].value_counts(normalize=True).to_dict()
test_dp_count = test_df['Dst Port'].value_counts(normalize=True).to_dict()

test_df['src_port_count'] = test_df['Src Port'].map(test_sp_count)
test_df['dst_port_count'] = test_df['Dst Port'].map(test_dp_count)

In [63]:
# standardScaler
from sklearn.preprocessing import StandardScaler

columns_to_scale = test_df.select_dtypes(include=['int64','float64']).columns.drop(['Src Port', 'Dst Port',
        'hour'])
normalizer = StandardScaler()
normalizer = normalizer.fit(test_df[columns_to_scale])
test_df[columns_to_scale] = normalizer.transform(test_df[columns_to_scale])
test_df

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flags,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,req_cnt_per_hour,res_cnt_per_hour,req_freq_per_hour,res_freq_per_hour,req_even_per_hour,res_even_per_hour,src_ip_count,dst_ip_count,src_port_count,dst_port_count
0,21.122.91.166,23801,30.239.23.123,53,TCP,......S.,2023-12-01 03:33:43,-0.318453,-0.146660,-0.026920,...,-0.731769,-0.577774,-0.055232,0.365439,1.009936,0.707060,-0.767215,-0.597005,-0.707336,0.185707
1,21.122.91.166,28413,30.239.23.123,53,TCP,......S.,2023-12-01 03:32:49,-0.318817,-0.146660,-0.026920,...,-0.731769,-0.577774,-0.055232,0.365439,1.009936,0.707060,-0.767215,-0.597005,-0.707238,0.185707
7,21.122.91.166,3121,30.239.23.123,53,TCP,......S.,2023-12-01 03:55:03,-0.320913,-0.146660,-0.026920,...,-0.731769,-0.577774,-0.055232,0.365439,1.009936,0.707060,-0.767215,-0.597005,-0.707336,0.185707
9,21.122.91.166,53746,30.239.23.123,53,TCP,......S.,2023-12-01 03:02:00,-0.318299,-0.146660,-0.026920,...,-0.731769,-0.577774,-0.055232,0.365439,1.009936,0.707060,-0.767215,-0.597005,-0.706842,0.185707
13,21.122.91.166,26592,30.239.23.123,53,TCP,......S.,2023-12-01 03:21:42,-0.318492,-0.146660,-0.026920,...,-0.731769,-0.577774,-0.055232,0.365439,1.009936,0.707060,-0.767215,-0.597005,-0.707139,0.185707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324951,82.101.43.185,54945,112.21.60.180,2323,TCP,......S.,2023-12-01 03:56:27,2.196684,-0.146660,-0.026920,...,-0.838678,-0.582232,-0.219729,-0.109612,-0.098187,-2.858722,-0.819634,-0.598889,-0.705558,-0.726846
324953,82.101.43.185,54945,172.0.89.185,2323,TCP,......S.,2023-12-01 04:22:34,3.215577,-0.150008,-0.026920,...,-0.840116,-0.582232,-0.017684,-0.109612,-2.791554,-2.858722,-0.819634,-0.598889,-0.705558,-0.726846
325203,82.38.47.203,54578,112.21.100.216,51877,TCP,......S.,2023-12-01 05:00:09,-0.437078,0.626708,-0.026920,...,-0.839636,-0.582232,0.449337,-0.109612,-2.791554,-2.858722,-0.743433,-0.597123,-0.627697,-0.729139
325401,82.38.47.203,54578,30.17.110.208,17598,TCP,......S.,2023-12-01 05:00:56,-0.437376,-0.150008,-0.026920,...,-0.839636,-0.582232,0.449337,-0.109612,-2.791554,-2.858722,-0.743433,-0.596613,-0.627697,-0.728546


In [64]:
test_df = pd.get_dummies(test_df, columns=['Protocol'])

In [65]:
test_df['Has_SYN'] = test_df['Flags'].apply(lambda x: 1 if 'S' in x else 0)
test_df['Has_FIN'] = test_df['Flags'].apply(lambda x: 1 if 'F' in x else 0)
test_df['Has_ACK'] = test_df['Flags'].apply(lambda x: 1 if 'A' in x else 0)
test_df['Has_RST'] = test_df['Flags'].apply(lambda x: 1 if 'R' in x else 0)
test_df['Has_PSH'] = test_df['Flags'].apply(lambda x: 1 if 'P' in x else 0)
test_df['Has_URG'] = test_df['Flags'].apply(lambda x: 1 if 'U' in x else 0)
test_df['Has_ECE'] = test_df['Flags'].apply(lambda x: 1 if 'E' in x else 0)
test_df['Has_CWR'] = test_df['Flags'].apply(lambda x: 1 if 'C' in x else 0)
test_df['Has_Not_Flags'] = test_df['Flags'].apply(lambda x: 1 if 'S' not in x and 'F' not in x and 'A' not in x and 'R' not in x and 'P' not in x and 'U' not in x and 'E' not in x and 'C' not in x else 0)

In [66]:
test_df['dst_port_same_src_port'] = (test_df['Dst Port'] == test_df['Src Port']).astype(int)

In [67]:
test_expt = test_df[[
       'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Std',
       'Packet Length Variance', 'Average Packet Size', 'Fwd Segment Size Avg',
       'Bwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'req_cnt_per_hour',
       'res_cnt_per_hour', 'req_freq_per_hour', 'res_freq_per_hour',
       'req_even_per_hour', 'res_even_per_hour', 'src_ip_count',
       'dst_ip_count','src_port_count',
       'dst_port_count', 'Protocol_TCP', 'Protocol_UDP',
       'Has_SYN', 'Has_FIN', 'Has_ACK', 'Has_RST', 'Has_PSH', 'Has_URG',
       'Has_ECE', 'Has_CWR', 'Has_Not_Flags', 'dst_port_same_src_port']]
test_list = test_expt.values.tolist()

In [68]:
#학습을 위한 데이터 타입 변환
test_X = np.array(test_list, dtype=np.float32)

In [69]:
y_pred_test = rfc.predict(test_X)

In [70]:
outer_test_all = open_pickle("./2023-2informationSecurity_testdataset/outer_IP_test_all.pkl")

In [71]:
predictIP_dict_test = {}

for idx, data in enumerate(test_df.values):
    if data[0] in outer_test_all:
        if data[0] not in predictIP_dict_test:
            predictIP_dict_test[data[0]] = 0

        if y_pred_test[idx] == 1:
            predictIP_dict_test[data[0]] = 1

    else:
        if data[2] not in outer_test_all:
            predictIP_dict_test[data[2]] = 0

        if y_pred_test[idx] == 1:
            predictIP_dict_test[data[2]] = 1

In [72]:
import csv
csv_file = '팀19_project2_result'
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    header = ["IP", "predict"]
    writer.writerow(header)

    for key, value in predictIP_dict_test.items():
        writer.writerow([key, value])